<a href="https://colab.research.google.com/github/Itsuki-Hamano123/practice-casualeffect/blob/master/PBC/Calculation_ATE_ATT_From_PBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# GoogleCoraboratoryで実験する場合にMyDriveをマウント
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# 対象ディレクトリに移動
cd /content/drive/My Drive/機械学習練習/統計的因果推論/因果推論実践用/PBC

/content/drive/My Drive/機械学習練習/統計的因果推論/因果推論実践用/PBC


In [3]:
ls

Calculation_ATE_ATT_From_PBC.ipynb  datasets/  memo.txt


In [0]:
import pandas as pd
import numpy as np

#原発性胆汁性胆管炎（Primary biliary cholangitis：PBC）に対してD-ペニシラミンを用いたA/Bテストの結果からD-ペニシラミンの（因果）効果測定を行う

手順

1.   DataPreprocessing
2.   （誤った方法で）A/Bテストを評価
3.   傾向スコアの導出
4.   IPWで因果効果ATEを求めA/Bテストを評価
5.   傾向スコアマッチングで因果効果ATTを求めA/Bテストを評価

*   介入変数（treatment）：drug（D-ペニシラミン）
*   アウトカム（outcome）：status（生存したか死亡したか）

**実験データのカラム説明**
* id       = case number
* futime   = number of days between registration and the earlier of death,transplantion, or study analysis time in July, 1986
* status   = **（アウトカム）**0=alive, 1=liver transplant, 2=dead
* drug     = **（介入変数）** 1= D-penicillamine, 2=placebo
* age      = age in days
* sex      = 0=male, 1=female
* ascites  = presence of ascites: 0=no 1=yes
* hepato   = presence of hepatomegaly 0=no 1=yes
* spiders  = presence of spiders 0=no 1=yes
* edema    = presence of edema 0=no edema and no diuretic therapy for edema; .5 = edema present without diuretics, or edema resolved by diuretics; 1 = edema despite diuretic therapy
* bili     = serum bilirubin in mg/dl
* chol     = serum cholesterol in mg/dl
* albumin  = albumin in gm/dl
* copper   = urine copper in ug/day
* alk_phos = alkaline phosphatase in U/liter
* sgot     = SGOT in U/ml
* trig     = triglicerides in mg/dl
* platelet = platelets per cubic ml/1000
* protime  = prothrombin time in seconds
* stage    = histologic stage of disease




In [5]:
DATASET_PATH = './datasets/analytical_data.csv'
EXPERIMENT_DATA_DF = pd.read_csv(DATASET_PATH)
TREATMENT_COLUM = 'drug'
OUTCOME_COLUM = 'status'
print("ExperimentDataSize:{data}".format(data=EXPERIMENT_DATA_DF.shape))

ExperimentDataSize:(418, 20)


In [6]:
EXPERIMENT_DATA_DF

,id,futime,status,drug,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk_phos,sgot,trig,platelet,protime,stage
0,1,400,2,1,21464,1,1,1,1,1.0,14.5,261,2.60,156,1718,137.95,172,190,12.2,4
1,2,4500,0,1,20617,1,0,1,1,0.0,1.1,302,4.14,54,7394.8,113.52,88,221,10.6,3
2,3,1012,2,1,25594,0,0,0,0,0.5,1.4,176,3.48,210,516,96.1,55,151,12,4
3,4,1925,2,1,19994,1,0,1,1,0.5,1.8,244,2.54,64,6121.8,60.63,92,183,10.3,4
4,5,1504,1,2,13918,1,0,1,1,0.0,3.4,279,3.53,143,671,113.15,72,136,10.9,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,414,681,2,.,24472,1,.,.,.,0.0,1.2,.,2.96,.,.,.,.,174,10.9,3
414,415,1103,0,.,14245,1,.,.,.,0.0,0.9,.,3.83,.,.,.,.,180,11.2,4
415,416,1055,0,.,20819,1,.,.,.,0.0,1.6,.,3.42,.,.,.,.,143,9.9,3
416,417,691,0,.,21185,1,.,.,.,0.0,0.8,.,3.75,.,.,.,.,269,10.4,3


#①DataPreprocessing

今回はATE求める練習のため前処理は簡単に以下処理のみ行う
*   D-penicillamineが投与された場合は1、placeboの場合は0に値変更
*   欠測値を含むユーザを除く
*   肝移植したユーザ（status=1）をのぞく
*   生存=0,死亡=2　⇒　生存=1,死亡=0に置き換え




In [0]:
def rewrite_categorical_binary_variable_value(data, stored_colum, flag_var, non_flag_var):
    """二値カテゴリ変数の書き換え

      WARNING:二値変数以外の値を取るカラムを渡して実行してはいけない

      Args:
         flag_val{category_name, current_val}(dict): 値を1にしたいカテゴリ {カテゴリ名, 現在格納されている値}
         non_flag_val{category_name, current_val}(dict): 値を0にしたいカテゴリ {カテゴリ名, 現在格納されている値}
    """
    data[stored_colum] = data[stored_colum].replace(flag_var['current_val'], flag_var['category_name']).replace(non_flag_var['current_val'], non_flag_var['category_name'])
    data[stored_colum] = data[stored_colum].replace(flag_var['category_name'], 1).replace(non_flag_var['category_name'], 0)
    return data

def remove_missing_value_line(data):
    """欠測値のある行を取り除く

     「.」のみまたは空白のデータが含まれる行を取り除く

    """
    for colum in data.columns.tolist():
        data.query("not " + colum + " in ['.', '']", inplace=True)
    preproccesing_data = data
    return preproccesing_data


def remove_status1_person(data):
    """肝移植したユーザを除く

     statusカラムが1となっている行を除く

    """
    preproccesing_data = data.query('not status == 1')
    return preproccesing_data

In [0]:
def remove_any_columns_data(data, remove_columlist=None):
    return data if remove_columlist == None else data.drop(remove_columlist, axis = 1)

def remove_element_from_list(target_list, expect_list):
    return list(set(target_list) ^ set(expect_list))

def df_change_type_to_double(data, expect_columlist=None):
    if expect_columlist == None:
       data = data.astype('float64')
    else:
       process_columns = remove_element_from_list(data.columns.tolist(), expect_columlist)
       data[process_columns] = data[process_columns].astype('float64')
    return data

def preproccesing_data(origin_data):
    """データ前処理用

    Args:
        origin_data(df): 整形前のデータ

    Returns:
        preprocessed_data(df): 整形後のデータ
    """
    stored_colum, flag_var, non_flag_var = TREATMENT_COLUM, {'category_name' : 'D-penicillamine', 'current_val' : '1'}, {'category_name' : 'placebo', 'current_val' : '2'}
    preproccessed_data = rewrite_categorical_binary_variable_value(origin_data, stored_colum, flag_var, non_flag_var)
    
    preproccessed_data = remove_missing_value_line(preproccessed_data)
    
    preproccessed_data = remove_status1_person(preproccessed_data)

    stored_colum_st, flag_var_st, non_flag_var_st = 'status', {'category_name' : 'dead', 'current_val' : 2}, {'category_name' : 'survive', 'current_val' : 0}
    preproccessed_data = rewrite_categorical_binary_variable_value(preproccessed_data.copy(), stored_colum_st, flag_var_st, non_flag_var_st)

    preproccessed_data = remove_any_columns_data(preproccessed_data, ['id'])

    preproccessed_data = df_change_type_to_double(preproccessed_data, ['drug', 'status'])

    return preproccessed_data

In [9]:
PREPROCESSED_DATA = preproccesing_data(EXPERIMENT_DATA_DF)
print("PreprocessedDataSize:{data}".format(data=PREPROCESSED_DATA.shape))

PreprocessedDataSize:(258, 19)


In [10]:
PREPROCESSED_DATA

,futime,status,drug,age,sex,ascites,hepato,spiders,edema,bili,chol,albumin,copper,alk_phos,sgot,trig,platelet,protime,stage
0,400.0,1,1,21464.0,1.0,1.0,1.0,1.0,1.0,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,4500.0,0,1,20617.0,1.0,0.0,1.0,1.0,0.0,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,1012.0,1,1,25594.0,0.0,0.0,0.0,0.0,0.5,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,1925.0,1,1,19994.0,1.0,0.0,1.0,1.0,0.5,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
6,1832.0,0,0,20284.0,1.0,0.0,1.0,0.0,0.0,1.0,322.0,4.09,52.0,824.0,60.45,213.0,204.0,9.7,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,1153.0,0,1,22347.0,1.0,0.0,1.0,0.0,0.0,0.4,246.0,3.58,24.0,797.0,91.00,113.0,288.0,10.4,2.0
308,994.0,0,0,21294.0,1.0,0.0,0.0,0.0,0.0,0.4,260.0,2.75,41.0,1166.0,70.00,82.0,231.0,10.8,2.0
309,939.0,0,1,22767.0,1.0,0.0,0.0,0.0,0.0,1.7,434.0,3.35,39.0,1713.0,171.00,100.0,234.0,10.2,2.0
310,839.0,0,1,13879.0,1.0,0.0,0.0,0.0,0.0,2.0,247.0,3.16,69.0,1050.0,117.00,88.0,335.0,10.5,2.0


In [0]:
def extract_any_category_data(data, target_categorical_name, target_categorical_num):
    extracted_data = data[data[target_categorical_name] == target_categorical_num]
    return extracted_data

In [12]:
TREATMENTGROUP_DATA = extract_any_category_data(PREPROCESSED_DATA, TREATMENT_COLUM, 1)
CONTROLGROUP_DATA = extract_any_category_data(PREPROCESSED_DATA, TREATMENT_COLUM, 0)
print(TREATMENTGROUP_DATA)
print(CONTROLGROUP_DATA)

     futime  status drug      age  sex  ...    sgot   trig  platelet  protime  stage
0     400.0       1    1  21464.0  1.0  ...  137.95  172.0     190.0     12.2    4.0
1    4500.0       0    1  20617.0  1.0  ...  113.52   88.0     221.0     10.6    3.0
2    1012.0       1    1  25594.0  0.0  ...   96.10   55.0     151.0     12.0    4.0
3    1925.0       1    1  19994.0  1.0  ...   60.63   92.0     183.0     10.3    4.0
8    2400.0       1    1  15526.0  1.0  ...  144.15   88.0     251.0     11.0    2.0
..      ...     ...  ...      ...  ...  ...     ...    ...       ...      ...    ...
301  1271.0       0    1  13806.0  1.0  ...   52.00  104.0     268.0     10.6    2.0
303  1230.0       0    1  12979.0  1.0  ...   45.00   75.0     246.0     10.8    3.0
307  1153.0       0    1  22347.0  1.0  ...   91.00  113.0     288.0     10.4    2.0
309   939.0       0    1  22767.0  1.0  ...  171.00  100.0     234.0     10.2    2.0
310   839.0       0    1  13879.0  1.0  ...  117.00   88.0     33

In [13]:
print("TreatmentGroupSize:{treated_size}".format(treated_size = TREATMENTGROUP_DATA.shape[0]))
print("ControlGroupSize:{control_size}".format(control_size=CONTROLGROUP_DATA.shape[0]))

TreatmentGroupSize:127
ControlGroupSize:131


In [14]:
pd.set_option('display.max_columns', 50)
print("TreatmentGroupStatistics:\n{treated_statistics}".format(treated_statistics = TREATMENTGROUP_DATA.describe()))
print("ControlGroupStatistics:\n{control_statistics}".format(control_statistics=CONTROLGROUP_DATA.describe()))

TreatmentGroupStatistics:
            futime      status           age         sex     ascites  \
count   127.000000  127.000000    127.000000  127.000000  127.000000   
mean   2001.039370    0.448819  18979.551181    0.866142    0.086614   
std    1097.222326    0.499343   3967.378697    0.341848    0.282383   
min      41.000000    0.000000   9598.000000    0.000000    0.000000   
25%    1234.500000    0.000000  16270.000000    1.000000    0.000000   
50%    1827.000000    0.000000  19155.000000    1.000000    0.000000   
75%    2673.000000    1.000000  21969.500000    1.000000    0.000000   
max    4556.000000    1.000000  28650.000000    1.000000    1.000000   

           hepato     spiders       edema        bili         chol  \
count  127.000000  127.000000  127.000000  127.000000   127.000000   
mean     0.472441    0.307087    0.129921    2.925984   353.543307   
std      0.501217    0.463113    0.290133    3.762708   180.429459   
min      0.000000    0.000000    0.000000    

#②（誤った方法で）A/Bテストを評価

In [0]:
def calc_naive_tau(outcome_colum_name, tratment_group_data, control_group_data):
    """　単純なoutcomeの差を求める

     WARNING:誤った効果測定の方法
　　 計算方法：介入群のoutcomeの平均値 - 対照群のoutcomeの平均値
    """
    naive_tau = tratment_group_data[outcome_colum_name].mean() - control_group_data[outcome_colum_name].mean()
    return naive_tau

In [16]:
naive_tau_hat = calc_naive_tau(OUTCOME_COLUM, TREATMENTGROUP_DATA, CONTROLGROUP_DATA)
print("naiveτ~ = {naive_tau_hat}".format(naive_tau_hat = naive_tau_hat))

naiveτ~ = 0.03660515717977997


# ③（因果推論手法を用いた）因果効果測定

## （準備）傾向スコアの算出
今回はLogistic Regression（ロジスティック回帰）を使い、傾向スコア（被験者の共変量に応じて介入群に割り振られる確率）を算出

In [0]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression

ss = preprocessing.StandardScaler()
lr = LogisticRegression(random_state = 7)

In [0]:
def process_standardization(data):
    return ss.fit_transform(data)

def preprocess_data_for_lr(data, extract_colum_list = None):
    """ロジスティック回帰用データ整形
    
    ①不要なカラムの削除
    ②データの標準化

    """
    processed_data = remove_any_columns_data(data, extract_colum_list)

    processed_data_np = process_standardization(processed_data)
    
    return processed_data_np

In [0]:
# トリートメント割り当て確率を予測するための特徴量
TRAINING_VECTOR_X= preprocess_data_for_lr(PREPROCESSED_DATA, [TREATMENT_COLUM, OUTCOME_COLUM])
#pd.DataFrame(TRAINING_VECTOR_X).describe()
#TRAINING_VECTOR_X

In [0]:
# モデルの学習